In [1]:
%load_ext Cython

In [1]:
import json
from datetime import datetime
import optuna as op
from optuna.samplers import TPESampler
%matplotlib inline
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Utils.recsys2022DataReader import createBumpURM

In [2]:
URM = createBumpURM()

/Users/matteopancini/PycharmProjects/recsys-challenge-2022-Pancini-Vitali/Utils/recsys2022DataReader.py:12: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(urmPath)


In [3]:
URM_train_list = []
URM_validation_list = []

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage=0.80)

for k in range(2):
    URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage=0.80)
    URM_train_list.append(URM_train)
    URM_validation_list.append(URM_validation)

recommender_SLIMElasticNet_list = []

In [4]:
def objective(trial):

    MAP_List_Fold = []

    alpha = trial.suggest_float("alpha", 0.00001, 0.1, log=True)
    l1_ratio = trial.suggest_float("l1_ratio", 0.000001, 0.1, log=True)
    topK = trial.suggest_int("topK", 300, 400, log=True)

    for index in range(len(URM_train_list)):
        recommender_SLIMElasticNet = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train_list[index], verbose=False)

        recommender_SLIMElasticNet.fit(alpha=alpha, l1_ratio=l1_ratio, topK=topK)

        recommender_SLIMElasticNet_list.append(recommender_SLIMElasticNet)

        evaluator_validation = EvaluatorHoldout(URM_validation_list[index], cutoff_list=[10])

        result_dict, _ = evaluator_validation.evaluateRecommender(recommender_SLIMElasticNet)

        MAP_List_Fold.append(result_dict.iloc[0]["MAP"])

    return sum(MAP_List_Fold) / len(MAP_List_Fold)

In [ ]:
study = op.create_study(direction='maximize', sampler=TPESampler(multivariate=True))
study.optimize(objective, n_trials=2)

/Users/matteopancini/opt/anaconda3/envs/recsys/lib/python3.9/site-packages/optuna/samplers/_tpe/sampler.py:281: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2022-11-10 12:02:43,870] A new study created in memory with name: no-name-30d7bfa6-0bf2-426d-98b7-cbe1d6b9dd9a
100%|█████████▉| 24504/24507 [08:02<00:00, 50.74it/s]


EvaluatorHoldout: Ignoring 706 ( 1.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 40923 (100.0%) in 21.09 sec. Users per second: 1940


100%|█████████▉| 24504/24507 [06:45<00:00, 60.41it/s]


EvaluatorHoldout: Ignoring 1509 ( 3.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 40120 (100.0%) in 20.65 sec. Users per second: 1943


[I 2022-11-10 12:18:14,685] Trial 0 finished with value: 0.013613480354096976 and parameters: {'alpha': 0.0065262478449697545, 'l1_ratio': 4.746396583333156e-05, 'topK': 350}. Best is trial 0 with value: 0.013613480354096976.
 30%|███       | 7432/24507 [02:17<04:16, 66.46it/s]

In [ ]:
alpha = study.best_params['alpha']
l1_ratio = study.best_params['l1_ratio']
topK = study.best_params['topK']

In [ ]:
op.visualization.plot_optimization_history(study)

In [ ]:
op.visualization.plot_param_importances(study)

In [33]:
recommender_SLIMElasticNet_list[0].fit(alpha=alpha, topK=topK)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
result_dict, _ = evaluator_test.evaluateRecommender(recommender_SLIMElasticNet_list[0])
result_dict.iloc[0]["MAP"]



  0%|          | 0/24507 [00:00<?, ?it/s]

  0%|          | 8/24507 [00:08<7:12:52,  1.06s/it]

  0%|          | 40/24507 [00:09<1:17:17,  5.28it/s]

  0%|          | 72/24507 [00:09<35:57, 11.33it/s]  

  0%|          | 104/24507 [00:15<49:32,  8.21it/s]

  1%|          | 136/24507 [00:16<34:53, 11.64it/s]

  1%|          | 168/24507 [00:18<33:04, 12.26it/s]

  1%|          | 200/24507 [00:20<30:09, 13.44it/s]

  1%|          | 232/24507 [00:24<37:12, 10.88it/s]

  1%|          | 264/24507 [00:24<26:30, 15.25it/s]

  1%|          | 296/24507 [00:28<31:55, 12.64it/s]

  1%|▏         | 328/24507 [00:31<33:38, 11.98it/s]

  1%|▏         | 360/24507 [00:31<25:06, 16.03it/s]

  2%|▏         | 392/24507 [00:33<26:05, 15.40it/s]

  2%|▏         | 424/24507 [00:38<37:12, 10.79it/s]

  2%|▏         | 456/24507 [00:39<28:15, 14.18it/s]

  2%|▏         | 488/24507 [00:41<27:16, 14.67it/s]

  2%|▏         | 520/24507 [00:45<35:21, 11.31it/s]

  2%|▏         | 552/24507 [00:46<27:19, 14.61it/s]


EvaluatorHoldout: Processed 41298 (100.0%) in 33.60 sec. Users per second: 1229


In [34]:
resultParameters = result_dict.to_json(orient="records")
parsed = json.loads(resultParameters)

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.057003,0.077355,0.067285,0.027259,0.036556,0.177157,0.083572,0.061718,0.36503,0.218124,...,0.992049,0.362127,0.992049,0.025624,9.118886,0.995109,0.049084,0.67286,3.703427,0.318744


In [ ]:
with open("logs/" + recommender_SLIMElasticNet_list[0].RECOMMENDER_NAME + "_logs_" + datetime.now().strftime('%b%d_%H-%M-%S') + ".json", 'w') as json_file:
    json.dump(study.best_params, json_file, indent=4)
    json.dump(parsed, json_file, indent=4)